Define the data path and read the FIF file and stimulus PC logs for pilot subject 2. While we're at it, tag each state in the dataframe with the start trigger that should have been sent.

In [5]:
import mne
import pandas as pd
import numpy as np
from pathlib import Path

from intermodulation.freqtag_spec import TRIGGERS

datapath = Path("C:/Users/gerce/Documents/Projects/results/syntax_im/241018/")
raw = mne.io.read_raw_fif(datapath / "pilot03.fif", preload=True)
logs2w = pd.read_pickle(datapath / "2024-10-18T16_2word_experiment.pkl")
logs1w = pd.read_pickle(datapath / "2024-10-18T16_1word_experiment.pkl")
statesdf = logs2w["states"]
contdf = logs2w["continuous"]

# No correct/incorrect label for queries, so we need to manually check the prev state
statesdf["query_corr"] = pd.NA
for i, row in statesdf.iterrows():
    if i == 0:
        continue
    if row["state"] == "query":
        prevwords = statesdf.loc[i - 1, ["word1", "word2"]]
        statesdf.loc[i, "query_corr"] = np.isin(row["word1"], prevwords)

# Assign a map of states to triggers
trigmap_base = {
    "fixation": pd.NA,
    "intertrial": pd.NA,
    "query": pd.NA,
    "pause": TRIGGERS.PAUSE,
}

trigmap2w = {
    "words": {
        "phrase": TRIGGERS.TWOWORD.PHRASE,
        "non-phrase": TRIGGERS.TWOWORD.NONPHRASE,
        "non-word": TRIGGERS.TWOWORD.NONWORD,
    },
    **trigmap_base,
}
trigmap1w = {
    "word": {
        "word": TRIGGERS.ONEWORD.WORD,
        "non-word": TRIGGERS.ONEWORD.NONWORD,
    },
    **trigmap_base,
}


def get_trig(row, trigmap):
    v = trigmap[row["state"]]
    if isinstance(v, dict):
        v = v[row["condition"]]
    return v


statesdf["trigger"] = statesdf.apply(get_trig, axis=1, trigmap=trigmap2w)

Opening raw data file C:\Users\gerce\Documents\Projects\results\syntax_im\241018\pilot03.fif...
    Read a total of 17 projection items:
        ssp_combined_0_68deg (1 x 306)  idle
        ssp_combined_1_68deg (1 x 306)  idle
        ssp_combined_2_68deg (1 x 306)  idle
        ssp_combined_3_68deg (1 x 306)  idle
        ssp_combined_4_68deg (1 x 306)  idle
        ssp_combined_5_68deg (1 x 306)  idle
        ssp_combined_6_68deg (1 x 306)  idle
        ssp_combined_7_68deg (1 x 306)  idle
        ssp_combined_8_68deg (1 x 306)  idle
        ssp_combined_9_68deg (1 x 306)  idle
        ssp_combined_10_68deg (1 x 306)  idle
        ssp_combined_11_68deg (1 x 306)  idle
        ssp_combined_12_68deg (1 x 306)  idle
        ssp_combined_13_68deg (1 x 306)  idle
        ssp_combined_14_68deg (1 x 306)  idle
        ssp_combined_15_68deg (1 x 306)  idle
        ssp_combined_16_68deg (1 x 306)  idle
    Range : 19000 ... 1687999 =     19.000 ...  1687.999 secs
Ready.
Opening raw data file 

C:\Users\gerce\AppData\Local\Temp\ipykernel_34916\2641766265.py:9: RuntimeWarning: This filename (C:\Users\gerce\Documents\Projects\results\syntax_im\241018\pilot03.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(datapath / "pilot03.fif", preload=True)


KeyError: 'pause'

Align the timestamps for the two datasets, given that clock drift is known to exist between the two sources